In [5]:
import chess
import chess.engine
import random
import numpy as np
from stockfish import Stockfish 
from numpy import load
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers 
import tensorflow.keras.callbacks as callbacks
import tensorflow as tf

In [7]:
model1 = models.load_model('model.h5')
model2 = models.load_model('model5.h5')

 # Repeated code from other notebook

In [2]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# example: h3 -> 17
def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # create empty 3d matrix for each color's pieces
    board3d = np.zeros((14, 8, 8), dtype=np.int8)
  # 1 or 0 for pieces's view on the matrix, 
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # addding attacks and valid moves, use chess package
  # can't have a dumb AI 
    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[12][i][j] = 1
        board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[13][i][j] = 1
    board.turn = aux

    return board3d

# Define MinMax Algorithm to calculate best moves

In [53]:
def NN_evaluate(board):
    board3d = split_dims(board)
    board3d = np.expand_dims(board3d, 0)
    return model.predict(board3d)[0][0]


def minimax(board, depth, alpha, beta, maximizing_player):
    line = []
    if depth == 0 or board.is_game_over(): 
        return NN_evaluate(board)
    moves = board.legal_moves
    
    if maximizing_player:
        max_eval = -np.inf
        for move in moves:
            board.push(move)
            current_eval = minimax(board, depth-1, alpha, beta, False)
            board.pop()
            max_eval = max(max_eval, current_eval)
            alpha = max(alpha, current_eval)
            if beta <= alpha:
                break
            return max_eval
    else: 
        min_eval = np.inf 
        for move in moves:
            board.push(move)
            current_eval = minimax(board, depth-1, alpha, beta, True)
            board.pop()
            min_eval = min(min_eval, current_eval)
            beta = min(beta, current_eval) 
            if beta <= alpha:
                break 
            return min_eval
    
def get_ai_move(board, depth):
    max_move = None
    max_eval = -np.inf   
    for move in board.legal_moves:
        board.push(move)
        current_eval = minimax(board, depth-1, -np.inf, np.inf, False)
        board.pop()
        if current_eval > max_eval:
            max_eval = current_eval
            max_move = move
    return max_move

In [21]:
def NN_evaluate(board):
    board3d = split_dims(board)
    board3d = np.expand_dims(board3d, 0)
    return model1.predict(board3d)[0][0]


def minimax(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over(): 
        return NN_evaluate(board)
    moves = board.legal_moves
    for move in moves: 
        if maximizing_player:
            score = -minimax(board, depth -1, -alpha, -beta, False)
        else:
            for move in moves:
                board.push(move)
                current_eval = -minimax(board, depth-1, -alpha, -beta, True)
                board.pop()
            if current_eval >= beta:
                return beta
            if current_eval > alpha:
                alpha = current_eval
        return alpha 
    
def get_ai_move(board, depth):
    max_move = None
    max_eval = -np.inf   
    for move in board.legal_moves:
        board.push(move)
        current_eval = minimax(board, depth-1, -np.inf, np.inf, False)
        board.pop()
        if current_eval > max_eval:
            max_eval = current_eval
            max_move = move
    return max_move

# Play vs AI using MinMax, will lose always 




In [23]:
board = chess.Board()

with chess.engine.SimpleEngine.popen_uci('C:\\Users\\coope\\Downloads\\Python\\Machine Learning\\Chess AI\\stockfish_15_win_x64_avx2\\stockfish_15_x64_avx2.exe') as engine:
    while True:
        move = get_ai_move(board, 1)
        board.push(move)
        print(f'\n{board}')
        if board.is_game_over():
            break

        move = engine.analyse(board, chess.engine.Limit(time=1), info=chess.engine.INFO_PV)['pv'][0]
        board.push(move)
        print(f'\n{board}')
        if board.is_game_over():
            break
        board


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R

r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R

r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . .
. . N . . N . .
P P P P P P P P
R . B Q K B . R

r n b q k b . r
p p p . p p p p
. . . . . n . .
. . . p . . . .
. . . . . . . .
. . N . . N . .
P P P P P P P P
R . B Q K B . R

r n b q k b . r
p p p . p p p p
. . . . . n . .
. . . p . . . .
. . . . . . P .
. . N . . N . .
P P P P P P . P
R . B Q K B . R

r n . q k b . r
p p p . p p p p
. . . . . n . .
. . . p . . . .
. . . . . . b .
. . N . . N . .
P P P P P P . P
R . B Q K B . R

r n . q k b . r
p p p . p p p p
. . . . . n . .
. . . p . . . .
. . . . . . b .
. . N . . N . B
P P P P P P . P
R . B Q K . . R

r n . q k b . r
p p p . p p p p
. . . . . n . .
. . . p . . . .
. . . . . . . .
. . N . . N . b


In [18]:
board = chess.Board()

while True:
    move = get_ai_move(board, 1) 
    board.push(move)
    print(f'\n{board}')
    if board.is_game_over():
        break
    
    move = get_ai_move(board, 1)
    board.push(move)
    print(f'\n{board}')
    if board.is_game_over():
        break
    board


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R

r . b q k b n r
p p p p p p p p
n . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R

r . b q k b n r
p p p p p p p p
n . . . . . . .
. . . . . . . .
. . . . . . . .
N . . . . N . .
P P P P P P P P
R . B Q K B . R

r . b q k b n r
p . p p p p p p
n . . . . . . .
. p . . . . . .
. . . . . . . .
N . . . . N . .
P P P P P P P P
R . B Q K B . R

r . b q k b n r
p . p p p p p p
n . . . . . . .
. p . . N . . .
. . . . . . . .
N . . . . . . .
P P P P P P P P
R . B Q K B . R

r . b q k b n r
p . p p p p . p
n . . . . . p .
. p . . N . . .
. . . . . . . .
N . . . . . . .
P P P P P P P P
R . B Q K B . R

r . b q k b n r
p . p p p p . p
n . N . . . p .
. p . . . . . .
. . . . . . . .
N . . . . . . .
P P P P P P P P
R . B Q K B . R

r . . q k b n r
p b p p p p . p
n . N . . . p .
. p . . . . . .
. . . . . . . .
N . . . . . . .



. . . . Q . n r
. . p p b . k .
. . N P . p . .
. P . . . . . p
. . . . . . . .
P . . P . . . .
R B . . P . P P
. . . . K B R .

. . . . Q . n .
. . p p b . k r
. . N P . p . .
. P . . . . . p
. . . . . . . .
P . . P . . . .
R B . . P . P P
. . . . K B R .

. . . . Q . n .
. . p p b . k r
. . N P . p . .
. P . . . . . p
. . . . . . . .
P . . P . . . .
R B . . P . P P
. . . . K B . R

. . . . Q . n r
. . p p b . k .
. . N P . p . .
. P . . . . . p
. . . . . . . .
P . . P . . . .
R B . . P . P P
. . . . K B . R

. . . . Q . n r
. . p p b . k .
. . N P . p . .
. P . . . . . p
. . . . . . . .
P . . P . . . .
R B . . P . P P
. . . . K B R .

. . . . Q . n .
. . p p b . k r
. . N P . p . .
. P . . . . . p
. . . . . . . .
P . . P . . . .
R B . . P . P P
. . . . K B R .

. . . . Q . n .
. . p p b . k r
. . N P . p . .
. P . . . . . p
. . . . . . . .
P . . P . . . .
R B . . P . P P
. . . . K B . R

. . . . Q . n r
. . p p b . k .
. . N P . p . .
. P . . . . . p
. . . . . . . .
P . . P . . . .


In [17]:
import chess.svg
import IPython

def play_game(board, depth):
    print(board)
    x = np.random.randint(100)
    if x > 50:
        print("You Start")
        while True:
            #Your move
            legal_moves = list(board.legal_moves)
            print(legal_moves)
            while True:
                move = input("Your move: ")
            try:
                board.push_uci(move)
                print(f'\n{board}')
                print('Turn', board.fullmove_number)
                break  
            except ValueError:
                print("Error: Invalid Move")
            if board.is_game_over():
                break
        
            #AI Move
            move = get_ai_move(board, depth)
            board.push(move)
            print(f'\n{board}')
            if board.is_game_over():
                break
    
    else:
        print("AI Starts")
        while True:
            #AI Move
            move = get_ai_move(board,depth)
            board.push(move)
            print(f'\n{board}')
            print('Turn', board.fullmove_number)
            if board.is_game_over():
                break
                
            #Your Move 
            legal_moves = list(board.legal_moves)
            print(legal_moves)
            while True:
                move = input("Your move: ")
            try:
                board.push_uci(move)
                print(f'\n{board}')
                print('Turn', board.fullmove_number)
                break  
            except ValueError:
                print("Error: Invalid Move")     
                
            if board.is_game_over():
                break    


In [18]:
board = chess.Board()
depth = 2
play_game(board, depth)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
AI Starts

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
Turn 1
[Move.from_uci('g8h6'), Move.from_uci('g8f6'), Move.from_uci('b8c6'), Move.from_uci('b8a6'), Move.from_uci('h7h6'), Move.from_uci('g7g6'), Move.from_uci('f7f6'), Move.from_uci('e7e6'), Move.from_uci('d7d6'), Move.from_uci('c7c6'), Move.from_uci('b7b6'), Move.from_uci('a7a6'), Move.from_uci('h7h5'), Move.from_uci('g7g5'), Move.from_uci('f7f5'), Move.from_uci('e7e5'), Move.from_uci('d7d5'), Move.from_uci('c7c5'), Move.from_uci('b7b5'), Move.from_uci('a7a5')]
Your move: g8h6


KeyboardInterrupt: Interrupted by user

In [31]:
ytest = np.load('originaldata\\boards3d.npy')
model2.predict(ytest)

array([[1.6285039e-05],
       [1.6285225e-05],
       [1.6285287e-05],
       ...,
       [1.6288968e-05],
       [1.6310061e-05],
       [1.6321048e-05]], dtype=float32)

In [17]:
ytrain = np.load('originaldata\\eval.npy')
model.
print(ytrain) 

AttributeError: module 'numpy' has no attribute 'normalize'